#Objective###
Given inputs, we want to find out if an observation is a star, galaxy, or quasar.

In [ ]:
import os
import sys
sys.path.append(os.path.abspath("./util/"))
from csv_data import SkyServerBinaryDatasetWrapper
import numpy as np

##Data##

###Data Loading###

Loading in Data using csv_data.py:

In [ ]:
# Loading data, the wrapper handles details of loading and data processing
wrapper = SkyServerBinaryDatasetWrapper()
[train_x, train_y], [valid_x, valid_y], [test_x, test_y] = wrapper.get_flat_datasets()

Downloading...
From: https://drive.google.com/u/0/uc?id=1gYwg5YyaV3zUX-07bLCol8E0M-uX__zX&export=download
To: /content/drive/My Drive/MLProjects/Astronomy Classification/util/skyserver.csv
100%|██████████| 1.39M/1.39M [00:00<00:00, 54.5MB/s]


###Examining Data###

Examining the shape of training data inputs (train_x), plus the first 5 samples:

In [ ]:
print(train_x.shape)
train_x[:5]

(7000, 13)


array([[ 0.16745842, -0.58492272,  1.03148637, -0.34855938, -0.83728027,
        -0.94605772, -0.99534154, -0.83806089,  0.21085172, -0.21763043,
        -0.36973112,  1.03148936,  1.30931064],
       [ 0.16886159, -0.58311429,  0.05243046, -0.16653251, -0.15415531,
        -0.08264457, -0.02604308, -0.83806089,  0.21085172, -0.21763043,
        -0.36984929, -0.63621258, -0.87919741],
       [ 0.17057433, -0.58347525,  0.92156796,  0.86709322,  0.59315368,
         0.44120145,  0.31452753, -0.83806089,  0.21085172, -0.21147922,
        -0.05302706, -0.65633905, -0.60919097],
       [ 0.17455754, -0.58650069, -1.03063038, -0.81362749, -0.63669227,
        -0.52660429, -0.43092107, -0.83806089,  0.21085172, -0.20532801,
        -0.36999261,  1.03148936,  1.30931064],
       [ 0.17482457, -0.58441247, -1.29023238, -1.17251944, -0.37676237,
        -0.02510121,  0.15827647, -0.83806089,  0.21085172, -0.20532801,
        -0.36818949,  1.03148936,  1.30931064]])

We can see there are 7000 samples with 13 features. These features have already been standardized for us.

Examining our training outputs/targets/ground truths (train_y):

In [ ]:
train_y[:5]

array([[0],
       [0],
       [1],
       [0],
       [0]])

There is a 0 when there is no star, and a 1 when there is a star.

##Method 1: Binary 0-1 Classification##
Here, we will determine a single value between 0 and 1 that gives us the probability we think an observation is in a SPECIFIC class. For example, if we are looking at whether an observation is a star, and our output is 0.7, this means the model believes there is a 70% chance the observation is a star.

###Choosing our NN Functions###

####NN Output Activation Function: Sigmoid####

We want to make our neural network output values between 0 and 1. Normally, output values can be anything on the continuous spectrum, but we can use activation functions to transform these values to a (0,1) range. We will use the sigmoid function to achieve this.

$\sigma=\frac{1}{1+ e^{-x}}$

The sigmoid uses this form because the $e$ in the sigmoid and the $log$ in the loss function will cancel out, making the gradient easier to calculate.

In [ ]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))

####Loss Function: Negative Log Likelihood (NLL)####

We cannot use Mean Squared Error (MSE) for our loss function in classification.

$MSE = (actual - predicted)^2$

Since our actual and predicted are constrained between 0 and 1, our MSE loss will be between 0 and 1. This does not allow for a steep gradient when performing gradient descent.

Instead, we use negative log likelihood(NLL):

$NLL = -(y * log(\hat{y}) + (1-y) * log(1-\hat{y}))$

**NOTE**: $log = log_e = ln$

Basically, we are taking our actual value and multiplying it by the log of our predicted value. Then, we add (1 - actual) * log(1-predicted). Since our actual is going to be either 0 or 1, intuitively one of these terms is going to always be 0.

This means we will ultimately get a value of the form $-log(z)$. The $log$ expression tells us what value $e$ needs to be raised to in order to equal $z$, and we multiply by $-1$ to make this value positive. Since $z$ will be between 0 and 1, the log function will always be negative.

**Note:** We generally add a small tolerance (e.g. 1e-6) to our predicted value to ensure our $log$ expression does not throw errors ($log(0)$ is undefined).

In [ ]:
tol = 1e-6
nll = lambda pred, actual: -((actual * np.log(pred + tol)) + ((1-actual) * np.log(1-pred + tol)))
# Below is the gradient of the loss function with respect to the output of the NN.
# Not going to explain how we calculate it, but it simplifies to (predicted - actual)
# thanks to our choice of the sigmoid activation function.
nll_grad = lambda pred, actual: pred - actual

###Defining the NN (Binary Classification)###

In [ ]:
#import network
#import activation
from dense import Dense

class ClassificationNet():
  def __init__(self, output_size=1):
    self.layer1= Dense(input_size=13, output_size=25)
    # We do not wish to apply ReLU function to the second layer
    self.layer2 = Dense(input_size=25, output_size=output_size, activation=False)

  def forward(self, x):
    x = self.layer1.forward(x)
    x = self.layer2.forward(x)
    return x

  def backward(self, grad, lr):
    grad = self.layer2.backward(grad, lr)
    self.layer1.backward(grad, lr)

In [ ]:
net = ClassificationNet(1)
# Hyperparameters
lr = 1e-2
epochs = 50

for epoch in range(epochs):
  epoch_loss = 0
  for x, target in zip(train_x, train_y):
    pred = sigmoid(net.forward(x.reshape(1,-1))) # The reshape turns the data into a column instead of a vector
    grad = nll_grad(pred, target)
    epoch_loss += nll(pred, target)[0,0]

    net.backward(grad, lr)

  if epoch % 10 == 0:
    print(f"Epoch: {epoch} train loss: {epoch_loss / len(train_x)}")
    epoch_loss = 0
    for x,target in zip(valid_x, valid_y):
      pred = sigmoid(net.forward(x.reshape(1, -1)))
      epoch_loss += nll(pred, target)[0, 0]
    print(f"Valid loss: {epoch_loss / len(valid_x)}")

Epoch: 0 train loss: 0.2682695827790348
Valid loss: 0.2644771902817305
Epoch: 10 train loss: 0.10104581465738757
Valid loss: 0.12039744183988187
Epoch: 20 train loss: 0.07648901768969217
Valid loss: 0.0931297334278475
Epoch: 30 train loss: 0.06804528798961347
Valid loss: 0.09299463791060265
Epoch: 40 train loss: 0.06247892193181192
Valid loss: 0.09788190123806989


##Method 2: Multiclass Classification##
Here, we will perform classification with multiple targets. We can extend our binary classification to get probabilities for $n$ classes. We need to adjust our target a little bit. One number is enough to differentiate between 2 classes (because it can be 0 or 1), but we need multiple numbers to differentiate between multiple classes.

###Multiclass Encoding###

Below we create a method that allows us to assign unique encodings for each class via 'One-Hot Encoding'.

In [ ]:
def encode(target, max_value=3):
  # Create a vector which has as many zeros as classes
  encoded = np.zeros((1, max_value))
  # We will set one of these indices to 1 (this is called One-Hot Encoding)
  encoded[0, target] = 1
  return encoded

Our encodings are as follows:

0 = Star => [[1, 0, 0]]

1 = Galaxy => [[0, 1, 0]]

2 = Quasar => [[0, 0, 1]]

**Note:** In the binary classification phase, we said that a 1 corresponded to a star, and a 0 corresponded to not being a star. We are using the same data, however due to the different context, in this scenario assume a 0 is a star and a 1 is a galaxy. We will not see any 2's as we are reusing the dataset meant for binary classification.

The output of our neural network will be three numbers, with each number corresponding to the probability that each object is associated with the code for its respective class.

**NOTE:** These numbers only become probabilities after we pass the outputs through the activation function discussed below. Before this occurs, the values can be anything (not necessarily bounded between 0 and 1).

###Choosing our NN Functions###

####NN Output Activation Function: Softmax####

We need two achieve two goals for the outputs of our NN:
1. The outputs need to fit into an appropriate range, e.g. (0,1)
2. All the outputs need to sum to 1

By using the sigmoid function, we could handle requirement #1. However, with three numbers, the sum $\sum_{k=0}^2 (output_k)$ can be greater than 1. Thus, we need to normalize the values across all three using a function that is **not** Sigmoid.

To normalize the values and keep them between 0 and 1, **instead of using the Sigmoid Function**, we can use a function called the Softmax Function:

$\zeta=\frac{e^{\hat{y_{i}}}}{\sum_{j=0}e^{\hat{y_{j}}}}$

In other words, the softmax function equals $e$ raised to the predicted y, divided by the sum of $e$ raised to all predicted y's.

Since we will be getting three numbers as our output from our NN, this is basically saying that for each value $z$ in an output [$z_1$, $z_2$, $z_3$], we apply the softmax function to that value.

In [ ]:
def softmax(preds):
  preds = np.exp(preds)
  if len(preds.shape) > 1: # This handles if preds is in a matrix
    # Sums across the rows, divides predictions by the sums
    # So each row in the matrix will be a separate set of predictions we want to softmax
    # The .reshape() will reshape the sum into a one column matrix, allowing us to broadcast across
    # all the columns/values of each output
    normed = preds / np.sum(preds, axis=1).reshape(-1,1)
  else: # This handles if preds is in a vector
    normed = preds / np.sum(preds)
  return normed

**Why does this make sense?**

We could use max() to find which value in the output is greatest, and assign probabilities of 1 for the max value's respective class, and 0's for the rest. For example, if we have an output of [10, 20, 30], we could take the max and use this to compute [0, 0, 1]. However, this does not give us good results to use for gradient descent, as we cannot compute $log(0)$. Instead, softmax pushes the maximum value towards 1, and it pushes the other values to a very small value approaching 0. It also normalizes the values so they add up to 1.

####Loss Function: Multiclass Negative Log Likelihood (NLL)####

We will be feeding into our loss function a vector such as [0.01, 0.01, 0.98], where each value is the probability the observation falls into its respective class.

The generalized negative log likelihood looks as follows:

$$NLL = - \sum_{i=0} y_{i} \log p_{i}$$

Before, when we did binary classification, we had one number, allowing us to compute for a single state. However, this can also be looked at in another way:
Say we want to perform binary classification on whether an observation is a star. Then in our loss function what we are really computing is the probability y is a star and that y is NOT a star. This second piece is how we got the $(1-y) * log(1 - \hat{y})$ expression in our binary NLL.

Just like in the binary case, however, the ground truth $y$ will only ever have a single 1 in its vector encoding, meaning while there is a summation, we are only ever computing this $y_i \log{p_i}$ term for the value in the predicted output corresponding to the correct class.

Ex: Say our ground truth $y$ for a particular observation is [0,1,0], i.e. a galaxy. If our prediction $\hat{y}$ is [0.01, 0.9, 0.09], we only end up computing the above term for the value 0.9 because the others are negated. So we are only looking at the loss against the ACTUAL $y$ value.

In [ ]:
def multiclass_loss(predicted, actual):
  tol = 1e-6
  # Cross Entropy is essentially how similar one distribution is to another
  cross_entropy = actual * np.log(predicted + tol)
  return -np.sum(cross_entropy)

In [ ]:
# Not going to explain how this is derived, but what happened with our binary NLL and sigmoid also
# occurs between our multiclass NLL and the softmax function
multiclass_loss_grad = lambda pred, actual: pred - actual

###Defining the NN (Multiclass Classification)###

We can actually reuse the ClassificationNet from before, except this time we want to specify our number of outputs to be 3, one for each class. Additionally, in our training loop we want to encode the targets (y values), and we want to use the softmax, multiclass_loss, and multiclass_loss_grad functions.

**NOTE:** Because we are using the data for the binary classification, each target (actual y value)will map to 0 or 1, meaning when we encode, we will get [1, 0, 0] or [0, 1, 0]. We will not have any [0, 0, 1] values in our data set, unfortunately.

In [ ]:
net = ClassificationNet(3)
# Hyperparameters
lr = 1e-3
epochs = 50

for epoch in range(epochs):
  epoch_loss = 0
  for x, target in zip(train_x, train_y):
    pred = softmax(net.forward(x.reshape(1,-1))) # The reshape turns the data into a column instead of a vector
    encoded = encode(target)
    grad = multiclass_loss_grad(pred, encoded)
    epoch_loss += multiclass_loss(pred, encoded)

    net.backward(grad, lr)

  if epoch % 10 == 0:
    print(f"Epoch: {epoch} train loss: {epoch_loss / len(train_x)}")
    epoch_loss = 0
    for x,target in zip(valid_x, valid_y):
      pred = softmax(net.forward(x.reshape(1, -1)))
      encoded = encode(target)
      epoch_loss += multiclass_loss(pred, encoded)
    print(f"Valid loss: {epoch_loss / len(valid_x)}")

Epoch: 0 train loss: 0.4525173340362456
Valid loss: 0.3038497893177284
Epoch: 10 train loss: 0.20493730457841222
Valid loss: 0.23215598386580852
Epoch: 20 train loss: 0.16465595194764043
Valid loss: 0.1957477666502035
Epoch: 30 train loss: 0.13581549486473626
Valid loss: 0.16032609837084882
Epoch: 40 train loss: 0.1172396942026143
Valid loss: 0.14132894594073564
